<a href="https://colab.research.google.com/github/AnanyaArora1812/AnanyaArora1812/blob/main/KDSH_Round1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



In [25]:
import os
os.listdir()

['.config',
 'results',
 'drive',
 'data',
 'In search of the castaways (1).txt',
 'The Count of Monte Cristo (1).txt',
 'results.csv',
 'sample_data']

In [26]:
with open("In search of the castaways (1).txt", "r", encoding="utf-8") as f:
    story = f.read()

print(len(story))
print(story[:1000])

826131
*** START OF THE PROJECT GUTENBERG EBOOK 46597 ***

Note: Project Gutenberg also has an HTML version of this
      file which includes the original 170 engravings.
      See 46597-h.htm or 46597-h.zip:
      (http://www.gutenberg.org/files/46597/46597-h/46597-h.htm)
      or
      (http://www.gutenberg.org/files/46597/46597-h.zip)


      Images of the original pages are available through
      HathiTrust Digital Library. See
      http://catalog.hathitrust.org/Record/008613392





A Voyage Round the World.

IN SEARCH OF THE CASTAWAYS:

A Romantic Narrative of the Loss of Captain Grant
of the Brig Britannia and of the Adventures of
His Children and Friends in His Discovery and Rescue.

[Illustration]

by

JULES VERNE,

Author of "Twenty Thousand Leagues Under the Sea," etc., etc.

Illustrated with One Hundred and Seventy Engravings







Philadelphia:
J. B. Lippincott & Co.
1874.




CONTENTS.

          I.      The Shark
         II.      The Three Documents
        III.     

In [27]:
with open("The Count of Monte Cristo (1).txt", "r", encoding="utf-8") as f:
    backstory = f.read()

print(len(backstory))
print(backstory[:1000])

2646614
﻿The Project Gutenberg eBook of The Count of Monte Cristo
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Count of Monte Cristo

Author: Alexandre Dumas
        Auguste Maquet

Release date: January 1, 1998 [eBook #1184]
                Most recently updated: August 9, 2025

Language: English

Credits: Anonymous Project Gutenberg Volunteers, Dan Muller and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE COUNT OF MONTE CRISTO ***




THE COUNT OF MONTE CRISTO

by Alexandre Dumas [père]




Contents


 VOLUME ONE
Chapter 1. Marseilles—The Arrival
Chapter 2. Father

In [29]:
import os
os.listdir()


['.config',
 'test (1).csv',
 'results',
 'drive',
 'data',
 'train.csv',
 'In search of the castaways (1).txt',
 'The Count of Monte Cristo (1).txt',
 'results.csv',
 'sample_data']

In [35]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test (1).csv")

print(train.head())
print(test.head())


    id                   book_name        char  \
0   46  In Search of the Castaways    Thalcave   
1  137   The Count of Monte Cristo       Faria   
2   74  In Search of the Castaways  Kai-Koumou   
3  109   The Count of Monte Cristo    Noirtier   
4  104   The Count of Monte Cristo    Noirtier   

                                             caption  \
0                                                NaN   
1  The Origin of His Connection with the Count of...   
2                                                NaN   
3         The Complexity of Family and Personal Life   
4  Involvement and Turning Point in the French Re...   

                                             content       label  
0  Thalcave’s people faded as colonists advanced;...  consistent  
1  Suspected again in 1815, he was re-arrested an...  contradict  
2  Before each fight he studied the crack-pattern...  consistent  
3  Villefort’s drift toward the royalists disappo...  contradict  
4  His parents were targete

In [33]:
print(train.columns)
print(test.columns)

Index(['id', 'book_name', 'char', 'caption', 'content', 'label'], dtype='object')
Index(['id', 'book_name', 'char', 'caption', 'content'], dtype='object')


In [37]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test (1).csv")

def clean_text(x):
    if pd.isna(x):
        return ""
    return str(x).lower().strip()

for col in ["book_name", "char", "caption", "content"]:
    train[col] = train[col].apply(clean_text)
    test[col] = test[col].apply(clean_text)

train["text"] = (
    "book: " + train["book_name"] +
    " | character: " + train["char"] +
    " | background: " + train["caption"] +
    " | statement: " + train["content"]
)

test["text"] = (
    "book: " + test["book_name"] +
    " | character: " + test["char"] +
    " | background: " + test["caption"] +
    " | statement: " + test["content"]
)

train[["text", "label"]].head()

,text,label
0,book: in search of the castaways | character: ...,consistent
1,book: the count of monte cristo | character: f...,contradict
2,book: in search of the castaways | character: ...,consistent
3,book: the count of monte cristo | character: n...,contradict
4,book: the count of monte cristo | character: n...,consistent


In [38]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train["label_enc"] = le.fit_transform(train["label"])

# Check mapping
print(dict(zip(le.classes_, le.transform(le.classes_))))

train[["label", "label_enc"]].head()

{'consistent': np.int64(0), 'contradict': np.int64(1)}


,label,label_enc
0,consistent,0
1,contradict,1
2,consistent,0
3,contradict,1
4,consistent,0


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Vectorize text
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2)
)

X_train = vectorizer.fit_transform(train["text"])
y_train = train["label_enc"]

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("Model trained successfully")

Model trained successfully


In [40]:
# Transform test text
X_test = vectorizer.transform(test["text"])

# Predict labels (numeric)
test_preds_enc = model.predict(X_test)

test_preds_enc[:5]

array([0, 0, 0, 0, 0])

In [41]:
# Convert encoded labels back to original labels
test_preds = le.inverse_transform(test_preds_enc)

test_preds[:5]

array(['consistent', 'consistent', 'consistent', 'consistent',
       'consistent'], dtype=object)

In [42]:
submission = pd.DataFrame({
    "id": test["id"],
    "label": test_preds
})

submission.head()

,id,label
0,95,consistent
1,136,consistent
2,59,consistent
3,60,consistent
4,124,consistent


In [43]:
submission.to_csv("submission.csv", index=False)
print("submission.csv saved")

submission.csv saved


In [44]:
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train, y_train)

print("Better model trained")

Better model trained


In [45]:
X_test = vectorizer.transform(test["text"])
test_preds_enc = model.predict(X_test)

In [46]:
test_preds = le.inverse_transform(test_preds_enc)

In [47]:
submission = pd.DataFrame({
    "id": test["id"],
    "label": test_preds
})

submission.to_csv("submission.csv", index=False)
print("submission.csv created")


submission.csv created
